In [131]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import random

weather_csv = gpd.read_file("./기상청_종관기상관측_ASOS.csv", encoding='cp949')
accident_csv = gpd.read_file("제주특별자치도_수난사고 구조출동 현황_2011_2020.csv")
length = len(accident_csv)
ran = int(random.random() * length)
print("coordinate : " + str(accident_csv['loc_info_x'][ran]) + ", " + str(accident_csv['loc_info_y'][ran]))

coordinate : 145459.04, 22572.9


In [2]:
'''
url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = '359e276820f89feab230a8eb1eeadedd'
redirect_uri = 'http://localhost:8888/' # APP에서 등록한 redirect_url
authorize_code = '-Ti1X-TEIwWEg8lHx3xTPOKZk5TdUQMW4M_joY99Vw_GMUHg78lt7g-McQx5fhPTvvUt0worDR4AAAGCFOaPpA'
 
data = {
        'grant_type':'authorization_code',
        'client_id':rest_api_key,
        'redirect_uri':redirect_uri,
        'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()
print(tokens["access_token"])
'''

NameError: name 'requests' is not defined

In [217]:
import requests

new_idx = []
new_x = []
new_y = []

for idx in range(0, length):  
    if(str(accident_csv['loc_info_y'][idx]) != ''):
        method = "GET"
        url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        params = {'x' : str(accident_csv['loc_info_x'][idx]), 
                  'y' : str(accident_csv['loc_info_y'][idx]),
                  'input_coord' : 'TM',
                  'output_coord' : 'WGS84'}
        header = {'authorization': 'KakaoAK 359e276820f89feab230a8eb1eeadedd'}

        response = requests.request(method=method, url=url, headers=header, params=params )
        tokens = response.json()

        x = tokens['documents'][0]['x']
        y = tokens['documents'][0]['y'] - 0.45

        new_idx.append(accident_csv['resc_reprt_no'])
        new_x.append(x)
        new_y.append(y)
    else: #has no coordinate
        print(idx)

88


In [212]:
#only one

import requests

method = "GET"
url = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
params = {'x' : 139516.8018, 
          'y' : 21483.4872,
          'input_coord' : 'WTM',
          'output_coord' : 'WGS84'}
header = {'authorization': 'KakaoAK 359e276820f89feab230a8eb1eeadedd'}

response = requests.request(method=method, url=url, headers=header, params=params )
tokens = response.json()

x = tokens['documents'][0]['x']
y = tokens['documents'][0]['y'] - 0.6

x,y
#input_coord 해본것들 WKTM, WTM(Y좌표만 어떻게하면?), TM (이것도 Y만), 

#126.3477 x좌표는 이건맞는듯
#WTM->WGS84 : (126.34771077228292, 33.685619937849175)
#CONGNAMUL->WGS84 : (125.44799755619425, 33.56415669858657)
#WCONGNAMUL->WGS84 : (125.44721978862324, 33.561386137382634)
#KTM->WGS84 : (125.21831790311661, 32.757107783355934)
#KTM->BESSEL : (125.22023064710514, 32.753776835065956)


(126.34771077228292, 33.085619937849174)

In [68]:
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 3.0 MB/s eta 0:00:00


In [218]:
import folium
m = folium.Map(location = [33.3734235,126.5512051,], zoom_start=8)
m.save('map.html')
m

In [219]:
for i in range(0, len(new_x)):
    folium.Marker(
        location = [new_y[i], new_x[i]],
        icon = folium.Icon(color="black", icon="star")).add_to(m)

m

In [173]:
folium.Marker(
    location = [y,x],
    icon = folium.Icon(color="blue", icon="star")).add_to(m)

m